# Basic tests of SAFTools + Sparseloop

## Eyeriss v2-like leader/follower skipping

In [1]:
!safinfer -h

usage: safinfer [-h] [-i DIR_IN] [-a ARCH] [-m MAP] [-p PROB] [-s SPARSEOPTS]
                [-o DIR_OUT] [-b BINDING_OUT] [-t TAXO_SCRIPT_LIB]
                [-T TOPOLOGY_OUT] [-r] [-L] [-f LOG_FILE] [-q SETTINGS]

optional arguments:
  -h, --help            show this help message and exit
  -i DIR_IN, --dir-in DIR_IN
  -a ARCH, --arch ARCH
  -m MAP, --map MAP
  -p PROB, --prob PROB
  -s SPARSEOPTS, --sparseopts SPARSEOPTS
  -o DIR_OUT, --dir-out DIR_OUT
  -b BINDING_OUT, --binding-out BINDING_OUT
  -t TAXO_SCRIPT_LIB, --taxo-script-lib TAXO_SCRIPT_LIB
  -T TOPOLOGY_OUT, --topology-out TOPOLOGY_OUT
  -r, --reconfigurable-arch
  -L, --log
  -f LOG_FILE, --log-file LOG_FILE
  -q SETTINGS, --settings SETTINGS
                        safinfer configuration file.


In [155]:
!safinfer -i src/ref_input/ -q src/ref_input/safinfer_settings.yaml -o src/ref_output/

- SAFinfer settings path: src/ref_input/safinfer_settings.yaml
logging: False
>> SAFinfer
:: Setup
:: => Done, setup
reconfigurable_arch: False
:: Taxonomic inference
- COMPLETE: microarchitecture inference
  => SUCCESS
:: => Done, taxonomic inference
<< Done, SAFinfer


In [5]:
!cat src/ref_output/new_arch.yaml.pretty

TestArchitecture(Architecture):
- Components:
  - BackingStorage(BufferStub)
  - iact_spad(BufferStub)
  - weight_spad(BufferStub)
  - psum_spad(BufferStub)
  - reg(BufferStub)
  - BackingStorage_datatype_format_uarch(FormatUarch)
  - iact_spad_datatype_format_uarch(FormatUarch)
  - weight_spad_datatype_format_uarch(FormatUarch)
  - reg_datatype_format_uarch(FormatUarch)
  - Skipping_iact_spad1_weight_spad0(SkippingUarch)
  - Skipping_weight_spad1_psum_spad1(SkippingUarch)
  - Skipping_iact_spad1_psum_spad0(SkippingUarch)
- Nets:
 - ['BackingStorage.md_out0', 'BackingStorage_datatype_format_uarch.md_in0']
 - ['BackingStorage.at_bound_in0', 'BackingStorage_datatype_format_uarch.at_bound_out0']
 - ['BackingStorage.md_out1', 'BackingStorage_datatype_format_uarch.md_in1']
 - ['BackingStorage.at_bound_in1', 'BackingStorage_datatype_format_uarch.at_bound_out1']
 - ['BackingStorage.md_out2', 'BackingStorage_datatype_format_uarch.md_in2']
 - ['BackingStorage.at_bound_in2', 'BackingStorage_data

In [6]:
!safmodel -h

usage: safmodel [-h] [-a ARCH] [-b CHAR] [-c COMP_IN] [-d RTL] [-f LOG_FILE]
                [-i DIR_IN] [-j HCL] [-k COMP_OUT] [-L] [-m MODEL_SCRIPT_LIB]
                [-T TOPOLOGY_OUT] [-o DIR_OUT] [-r ARCH_OUT] [-s SPARSEOPTS]
                [-t TAXO_SCRIPT_LIB] [-U SETTINGS] [-u SIM]

safmodel: SAF microarchitecture modeling generator.

optional arguments:
  -h, --help            show this help message and exit
  -a ARCH, --arch ARCH  Sparseloop architecture file.
  -b CHAR, --char CHAR  CSV EAT characterization table.
  -c COMP_IN, --comp-in COMP_IN
                        One or more Accelergy component library YAML
                        filenames.
  -d RTL, --rtl RTL     Path to underlying RTL files associated with CSV EAT
                        characterization table (-b/--char)
  -f LOG_FILE, --log-file LOG_FILE
                        Log filepath (requires -L/--log to enable logging).
  -i DIR_IN, --dir-in DIR_IN
                        Input files' directory. If not s

In [156]:
!safmodel -T src/ref_output/new_arch.yaml -a src/ref_input/arch.yaml -s src/ref_input/sparseopts.yaml -c src/ref_input/compound_components.yaml -U src/ref_input/safmodel_settings.yaml  -r src/ref_output/arch_w_SAF.yaml -k   src/ref_output/ -L

SAFmodel.

Parsing input files:
- netlist: src/ref_output/new_arch.yaml
- arch: src/ref_input/arch.yaml
- sparseopts: src/ref_input/sparseopts.yaml
- compound components (input): ['src/ref_input/compound_components.yaml']
- arch output path: src/ref_output/arch_w_SAF.yaml
- compound components path (output): src/ref_output/
- SAFModel settings path: src/ref_input/safmodel_settings.yaml
logging: True
>> SAFmodel
:: Setup
:: Scale inference
Building scale inference problem...
Building RegisteredEqualityComparator_bitwidth$(u) energy area latency table.: 1
- RegisteredEqualityComparator_bitwidth$(u) MSE energy error (symbolic vs numerical): 1.4802147286861898e-31
- RegisteredEqualityComparator_bitwidth$(u) RNMSE energy error (numerical vs true): 8.4 %
- RegisteredEqualityComparator_bitwidth$(u) MSE area error (symbolic vs numerical): 1.848202831402923e-24
- RegisteredEqualityComparator_bitwidth$(u) RNMSE area error (numerical vs true): 2.6 %
Building RegisteredEqualityComparator_bitwidth$

Building RegisteredAdder_bitwidth$(u) energy area latency table.: 100%|█| 7/7 [0
- RegisteredAdder_bitwidth$(u) MSE energy error (symbolic vs numerical): 8.091175612432276e-31
- RegisteredAdder_bitwidth$(u) RNMSE energy error (numerical vs true): 0.6 %
- RegisteredAdder_bitwidth$(u) MSE area error (symbolic vs numerical): 3.2609616555664098e-25
- RegisteredAdder_bitwidth$(u) RNMSE area error (numerical vs true): 0.7 %
=> Done, build.
Solving scale inference problem.
  Termination condition: Optimal
    Objective value: 131.79232363007497
=> done, solve.
:: => Done, scale inference
:: Export Accelergy models
Exporting backend modeling suite.
=> done, export modeling suite.
:: => Done, Accelergy export
<< Done, SAFmodel


In [11]:
print("Summary of MINLP solver output:\n\n")
!sed -n '/INFO:root:  Raw solver solution output:/,/INFO:root:-- Populating analytical primitive model attributes.../p' safmodel.log

Summary of MINLP solver output:


INFO:root:  Raw solver solution output:
INFO:root:    Objective: (0.5*x6*(3.39519598264422e-06*x37**3.0 - 0.0004030131448777*x37**2.0 + 0.0196580319453973*x37 + 0.0203547110119046) + 0.3333333333333333*x19*(3.39519598264422e-06*x45**3.0 - 0.0004030131448777*x45**2.0 + 0.0196580319453973*x45 + 0.0203547110119046))*(x38*(0.00166193389941796*x5**3.0 - 0.19617144255632*x5**2.0 + 18.9034258453812*x5 + 12.6194203869047) + x15*(0.00166193389941796*x32**3.0 - 0.19617144255632*x32**2.0 + 18.9034258453812*x32 + 12.6194203869047) + x7*(0.00166193389941796*x4**3.0 - 0.19617144255632*x4**2.0 + 18.9034258453812*x4 + 12.6194203869047) + x30*(0.00166193389941796*x8**3.0 - 0.19617144255632*x8**2.0 + 18.9034258453812*x8 + 12.6194203869047) + x16*(0.00166193389941796*x14**3.0 - 0.19617144255632*x14**2.0 + 18.9034258453812*x14 + 12.6194203869047) + x40*(0.00166193389941796*x29**3.0 - 0.19617144255632*x29**2.0 + 18.9034258453812*x29 + 12.6194203869047) + x9*(0.001661933899

In [14]:
print("Summary of Accelergy primitive ERT/ART tables output by SAFModel\n\n")
!sed -n '/WARNING:root:OVERVIEW ACCELERGY TABLES:/,/WARNING:root:-- => done, generating backend primitive objective data structure/p' safmodel.log

Summary of Accelergy primitive ERT/ART tables output by SAFModel


INFO:root:  Accelergy ART
INFO:root:    MetadataParser :
INFO:root:

 clock = 5ns
technology = 45nm
area_multiplier = 1.0
format = U
strategy = sentinel
coordinate_arithmetic = no_arithmetic
md_in_rw_thresh = 16.0
md_in_pr_thresh = 1.0
md_in_ww_thresh = 16.0
 = 271.661625870602 um^2

INFO:root:

 clock = 5ns
technology = 45nm
area_multiplier = 1.0
format = U
strategy = sentinel
coordinate_arithmetic = no_arithmetic
md_in_rw_thresh = 32.0
md_in_pr_thresh = 1.0
md_in_ww_thresh = 1.0
 = 31.3283367236290 um^2

INFO:root:

 clock = 5ns
technology = 45nm
area_multiplier = 1.0
format = U
strategy = sentinel
coordinate_arithmetic = no_arithmetic
md_in_rw_thresh = 4.0
md_in_pr_thresh = 1.0
md_in_ww_thresh = 4.0
 = 85.2007444570911 um^2

INFO:root:

 clock = 5ns
technology = 45nm
area_multiplier = 1.0
format = C
strategy = sentinel
coordinate_arithmetic = no_arithmetic
md_in_rw_thresh = 32.0
md_in_pr_thresh = 1.0
md_in_ww_thresh 

In [18]:
!timeloop-model -h

ERROR: Input configuration file does not end with .cfg, .yml, or .yaml


In [157]:
!timeloop-model src/ref_input/arch.yaml src/ref_input/compound_components.yaml src/ref_input/map.yaml src/ref_input/prob.yaml src/ref_input/sparseopts.yaml -o src/ref_output/

execute:/usr/local/bin/accelergy src/ref_input/arch.yaml src/ref_input/compound_components.yaml src/ref_input/map.yaml src/ref_input/prob.yaml src/ref_input/sparseopts.yaml --oprefix timeloop-model. -o src/ref_output// > timeloop-model.accelergy.log 2>&1
Utilization = 1.00 | pJ/Algorithmic-Compute =    2.906 | pJ/Compute =    7.654


In [158]:
!timeloop-model src/ref_output/arch_w_SAF.yaml src/ref_output/compound_components.yaml src/ref_input/compound_components.yaml src/ref_input/map.yaml src/ref_input/prob.yaml src/ref_input/sparseopts.yaml -o src/ref_output/

execute:/usr/local/bin/accelergy src/ref_output/arch_w_SAF.yaml src/ref_output/compound_components.yaml src/ref_input/compound_components.yaml src/ref_input/map.yaml src/ref_input/prob.yaml src/ref_input/sparseopts.yaml --oprefix timeloop-model. -o src/ref_output// > timeloop-model.accelergy.log 2>&1
Utilization = 1.00 | pJ/Algorithmic-Compute =   11.643 | pJ/Compute =   30.671


## ExTensor-like bidirectional skipping with naive two-fingered-merge-based intersection

In [127]:
!safinfer -h

usage: safinfer [-h] [-i DIR_IN] [-a ARCH] [-m MAP] [-p PROB] [-s SPARSEOPTS]
                [-o DIR_OUT] [-b BINDING_OUT] [-t TAXO_SCRIPT_LIB]
                [-T TOPOLOGY_OUT] [-r] [-L] [-f LOG_FILE] [-q SETTINGS]

optional arguments:
  -h, --help            show this help message and exit
  -i DIR_IN, --dir-in DIR_IN
  -a ARCH, --arch ARCH
  -m MAP, --map MAP
  -p PROB, --prob PROB
  -s SPARSEOPTS, --sparseopts SPARSEOPTS
  -o DIR_OUT, --dir-out DIR_OUT
  -b BINDING_OUT, --binding-out BINDING_OUT
  -t TAXO_SCRIPT_LIB, --taxo-script-lib TAXO_SCRIPT_LIB
  -T TOPOLOGY_OUT, --topology-out TOPOLOGY_OUT
  -r, --reconfigurable-arch
  -L, --log
  -f LOG_FILE, --log-file LOG_FILE
  -q SETTINGS, --settings SETTINGS
                        safinfer configuration file.


In [159]:
root_dir='src/case_studies/cross_design_impact'
proj_dir=root_dir+'/fig1_format_comparison_setup'
! echo safinfer -a {proj_dir}/arch/arch.yaml -s {proj_dir}/sparse-opt/coordinate_list.yaml -q {root_dir}/safinfer_settings.yaml -o {root_dir}/output/

safinfer -a src/case_studies/cross_design_impact/fig1_format_comparison_setup/arch/arch.yaml -s src/case_studies/cross_design_impact/fig1_format_comparison_setup/sparse-opt/coordinate_list.yaml -q src/case_studies/cross_design_impact/safinfer_settings.yaml -o src/case_studies/cross_design_impact/output/


In [160]:
!safinfer -a {proj_dir}/arch/arch.yaml -s {proj_dir}/sparse-opt/coordinate_list.yaml -q {root_dir}/safinfer_settings.yaml -o {root_dir}/output/

- SAFinfer settings path: src/case_studies/cross_design_impact/safinfer_settings.yaml
logging: False
>> SAFinfer
:: Setup
:: => Done, setup
reconfigurable_arch: False
:: Taxonomic inference
- COMPLETE: microarchitecture inference
  => SUCCESS
:: => Done, taxonomic inference
<< Done, SAFinfer


In [147]:
!safmodel -h

usage: safmodel [-h] [-a ARCH] [-b CHAR] [-c COMP_IN] [-d RTL] [-f LOG_FILE]
                [-i DIR_IN] [-j HCL] [-k COMP_OUT] [-L] [-m MODEL_SCRIPT_LIB]
                [-T TOPOLOGY_OUT] [-o DIR_OUT] [-r ARCH_OUT] [-s SPARSEOPTS]
                [-t TAXO_SCRIPT_LIB] [-U SETTINGS] [-u SIM]

safmodel: SAF microarchitecture modeling generator.

optional arguments:
  -h, --help            show this help message and exit
  -a ARCH, --arch ARCH  Sparseloop architecture file.
  -b CHAR, --char CHAR  CSV EAT characterization table.
  -c COMP_IN, --comp-in COMP_IN
                        One or more Accelergy component library YAML
                        filenames.
  -d RTL, --rtl RTL     Path to underlying RTL files associated with CSV EAT
                        characterization table (-b/--char)
  -f LOG_FILE, --log-file LOG_FILE
                        Log filepath (requires -L/--log to enable logging).
  -i DIR_IN, --dir-in DIR_IN
                        Input files' directory. If not s

In [149]:
!echo safmodel -T {root_dir}/output/new_arch.yaml -a {proj_dir}/arch/arch.yaml -s {proj_dir}/sparse-opt/coordinate_list.yaml -c src/ref_input/compound_components.yaml -U {root_dir}/safmodel_settings.yaml  -r {root_dir}/output/arch_w_SAF.yaml -k {root_dir}/output/ -L

safmodel -T src/case_studies/cross_design_impact/output/new_arch.yaml -a src/case_studies/cross_design_impact/fig1_format_comparison_setup/arch/arch.yaml -s src/case_studies/cross_design_impact/fig1_format_comparison_setup/sparse-opt/coordinate_list.yaml -c src/ref_input/compound_components.yaml -U src/case_studies/cross_design_impact/safmodel_settings.yaml -r src/case_studies/cross_design_impact/output/arch_w_SAF.yaml -k src/case_studies/cross_design_impact/output/ -L


In [163]:
!safmodel -T {root_dir}/output/new_arch.yaml -a {proj_dir}/arch/arch.yaml -s {proj_dir}/sparse-opt/coordinate_list.yaml -c src/ref_input/compound_components.yaml -U {root_dir}/safmodel_settings.yaml  -r {root_dir}/output/arch_w_SAF.yaml -k {root_dir}/output/ -L

SAFmodel.

Parsing input files:
- netlist: src/case_studies/cross_design_impact/output/new_arch.yaml
- arch: src/case_studies/cross_design_impact/fig1_format_comparison_setup/arch/arch.yaml
- sparseopts: src/case_studies/cross_design_impact/fig1_format_comparison_setup/sparse-opt/coordinate_list.yaml
- compound components (input): ['src/ref_input/compound_components.yaml']
- arch output path: src/case_studies/cross_design_impact/output/arch_w_SAF.yaml
- compound components path (output): src/case_studies/cross_design_impact/output/
- SAFModel settings path: src/case_studies/cross_design_impact/safmodel_settings.yaml
logging: True
>> SAFmodel
:: Setup
:: Scale inference
Building scale inference problem...
Building RegisteredEqualityComparator_bitwidth$(u) energy area latency table.: 1
- RegisteredEqualityComparator_bitwidth$(u) MSE energy error (symbolic vs numerical): 1.4802147286861898e-31
- RegisteredEqualityComparator_bitwidth$(u) RNMSE energy error (numerical vs true): 8.4 %
- Regi

In [ ]:
!timeloop-model src/ref_output/arch_w_SAF.yaml src/ref_output/compound_components.yaml src/ref_input/compound_components.yaml src/ref_input/map.yaml src/ref_input/prob.yaml src/ref_input/sparseopts.yaml -o src/ref_output/